In [1]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우의 한글 폰트
plt.rcParams['axes.unicode_minus'] = False       # 마이너스 기호 깨짐 방지

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 처리
import pandas as pd
import numpy as np

train_df = pd.read_csv("/data/train.csv")
test_df = pd.read_csv("/data/test.csv")

# 새로운 조합 변수 추가
for df in [train_df, test_df]:
    df['신용한도_대비_미상환율'] = df['현재 미상환 신용액'] / (df['최대 신용한도'] + 1)
    df['연체_빈도율'] = df['신용 문제 발생 횟수'] / (df['신용 거래 연수'] + 1)
    df['소득_대비_월_부채'] = df['월 상환 부채액'] / (df['연간 소득'] / 12 + 1)
    df['부채비율'] = df['현재 대출 잔액'] / (df['연간 소득'] + 1)
    df['개인파산_영향력'] = df['개인 파산 횟수'] * df['신용 문제 발생 횟수']


# "UID" 컬럼 유지 (제출 파일용)
test_uid = test_df["UID"]

# "UID" 컬럼 삭제
train_df.drop(columns=["UID"], inplace=True)
test_df.drop(columns=["UID"], inplace=True)

# X, y 분리
X = train_df.drop(columns=["채무 불이행 여부"])
y = train_df["채무 불이행 여부"]

In [3]:
# def convert_years(val):
#     if val == '1년 미만':
#         return 0
#     elif val == '10년 이상':
#         return 10
#     else:
#         # '4년', '6년' 등에서 '년'을 제거하고 정수형으로 변환
#         return int(val.replace('년', ''))

# # train, test 데이터에 대해 변환 적용
# X['현재 직장 근속 연수'] = X['현재 직장 근속 연수'].apply(convert_years)
# test_df['현재 직장 근속 연수'] = test_df['현재 직장 근속 연수'].apply(convert_years)

# # 변환 결과 확인
# print(X['현재 직장 근속 연수'].unique())


# 현재 직장 근속 연수를 이진 변수로 변환 (1년 미만: 1, 1년 이상: 0)
X['근속연수_1년미만'] = (X['현재 직장 근속 연수'] == '1년 미만').astype(int)
test_df['근속연수_1년미만'] = (test_df['현재 직장 근속 연수'] == '1년 미만').astype(int)

# 기존 변수 제거
X.drop(columns=['현재 직장 근속 연수'], inplace=True)
test_df.drop(columns=['현재 직장 근속 연수'], inplace=True)


In [4]:
# 원-핫 인코딩 적용
X = pd.get_dummies(X, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)
test_df = pd.get_dummies(test_df, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)

In [5]:
# from sklearn.model_selection import KFold

# # 타깃 인코딩을 적용할 범주형 변수 목록
# target_encoding_cols = ['주거 형태', '대출 목적', '대출 상환 기간']

# kf = KFold(n_splits=5, shuffle=True, random_state=31)

# for col in target_encoding_cols:
#     target_mean_map = {}

#     for train_idx, val_idx in kf.split(X):
#         train_fold, val_fold = X.iloc[train_idx], X.iloc[val_idx]
#         y_train_fold = y.iloc[train_idx]  # 🔹 y도 동일한 인덱싱으로 분할

#         # 🔹 타깃 평균 계산 (X_train과 y를 합쳐서 groupby)
#         train_fold = train_fold.copy()
#         train_fold['y'] = y_train_fold  # 🔥 여기서 y 값을 추가!

#         mean_encoding = train_fold.groupby(col)['y'].mean().to_dict()
#         target_mean_map.update(mean_encoding)

#     # 새로운 타깃 인코딩 컬럼 추가
#     X[col + '_타깃인코딩'] = X[col].map(target_mean_map)
#     test_df[col + '_타깃인코딩'] = test_df[col].map(target_mean_map)

# # 원본 범주형 컬럼 삭제
# X.drop(columns=target_encoding_cols, inplace=True)
# test_df.drop(columns=target_encoding_cols, inplace=True)

# print("✅ 타깃 인코딩 적용 완료!")


In [6]:
# # 중요도가 낮은 "대출 목적" 컬럼들 제거
# drop_loan_purpose_cols = [
#     '대출 목적_사업 대출', '대출 목적_여행 자금', 
#     '대출 목적_주택 개보수', '대출 목적_주택 구매'
# ]

# # 변수 삭제
# X.drop(columns=drop_loan_purpose_cols, inplace=True)
# test_df.drop(columns=drop_loan_purpose_cols, inplace=True)


In [7]:
# 로그 변환
log_columns = ["현재 미상환 신용액", "월 상환 부채액", "현재 대출 잔액", "연간 소득",
               '신용한도_대비_미상환율', '소득_대비_월_부채', '부채비율'] # 최대 신용한도 컬럼도 추가 가능
for col in log_columns:
    X[col] = np.log1p(X[col])
    test_df[col] = np.log1p(test_df[col])

# "마지막 연체 이후 경과 개월 수"가 0이면 "연체 없음" 컬럼 추가
# X["연체 없음"] = (X["마지막 연체 이후 경과 개월 수"] == 0).astype(int)
# test_df["연체 없음"] = (test_df["마지막 연체 이후 경과 개월 수"] == 0).astype(int)

In [8]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)
test_imputed = imputer.transform(test_df)
print(X_imputed.shape)
print(test_imputed.shape)

(10000, 35)
(2062, 35)


In [9]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_imputed)
# test_scaled = scaler.transform(test_imputed)

In [10]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 클래스 가중치 계산

classes = np.unique(y)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y)
class_weight_dict = dict(zip(classes, class_weights))

In [12]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tensorflow.keras.metrics import AUC

# 모델 생성 함수 수정
def create_model(dropout_rate=0.2, l2_reg=0.01):
    model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(l2_reg)),
        keras.layers.Dropout(dropout_rate),
        keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(l2_reg)),
        keras.layers.Dropout(dropout_rate),
        keras.layers.Dense(1, activation='sigmoid')  # 이진 분류
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name="auc")])  # AUC 이름 고정
    return model

# 하이퍼파라미터 후보군
dropout_rates = [0.2, 0.3, 0.4]
l2_regs = [0.001, 0.01, 0.1]

# StratifiedKFold 설정 (5폴드)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_auc = 0
best_params = None

# 하이퍼파라미터 조합별로 KFold 검증
for dropout in dropout_rates:
    for l2 in l2_regs:
        print(f"\n🔍 [교차 검증 시작] dropout: {dropout}, l2_reg: {l2}")

        fold_aucs = []  # 각 Fold의 AUC 저장
        for train_idx, val_idx in cv.split(X_scaled, y):
            X_train, X_val = X_imputed[train_idx], X_imputed[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            model = create_model(dropout, l2)

            # 조기 종료 설정 (이제 "val_auc"가 정확하게 고정됨)
            early_stopping = keras.callbacks.EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True)

            # 모델 학습
            history = model.fit(
                X_scaled, y,
                epochs=30,
                batch_size=32,
                validation_split=0.2,
                class_weight=class_weight_dict,  # 클래스 가중치 적용!
                callbacks=[early_stopping]
            )


            # 검증 데이터 AUC 평가
            val_auc = model.evaluate(X_val, y_val, verbose=0)[1]
            fold_aucs.append(val_auc)

        # 평균 AUC 계산
        mean_auc = np.mean(fold_aucs)
        print(f"✅ [교차 검증 완료] dropout: {dropout}, l2: {l2}, 평균 AUC: {mean_auc}")

        # 최고 AUC 갱신
        if mean_auc > best_auc:
            best_auc = mean_auc
            best_params = (dropout, l2)

print(f"\n🎯 최고 AUC: {best_auc} (dropout: {best_params[0]}, l2: {best_params[1]})")

TypeError: object of type 'NoneType' has no len()

In [ ]:
# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'units': [64, 128],
#     'dropout': [0.2, 0.5],
#     'l2_reg': [0.0001, 0.001],
#     'learning_rate': [0.001, 0.01],
#     'batch_size': [32, 64],
#     'epochs': [50]  # 조기 종료가 있으므로 크게 설정
# }
